# Efficient Yelp API Calls (Core)

# Assigment 

- For this assignment, you will be working with the Yelp API.

- As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

- Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

- Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
# Load API Credentials
with open('/Users/Justice/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Seafood'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
FILE = "Data/results_in_progress_NY_seafood.json"
FILE

'Data/results_in_progress_NY_seafood.json'

In [9]:
def create_json_file(FILE, delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists == True:
            
            print(f"[!] {FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exists.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(FILE, delete_if_exists=False)
        else:
            print(f"[i] {FILE} already exists.")
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {FILE} not found. Saving empty list to a new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        
        ## Save an empty list to start the json file
        with open(FILE, 'w') as f:
            json.dump([], f)


In [10]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(FILE, delete_if_exists=True)

## Load previous results and use len of results for offset
with open(FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

## How many results total?
total_results = results['total']

## How many did we get the details for?
results_per_page = len(results['businesses'])

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_seafood.json not found. Saving empty list to a new file.
- 0 previous results found.


805

In [11]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/805 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [12]:
# load final results
seafood_df = pd.read_json(FILE)
display(seafood_df.head(), seafood_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,YvWQMlMjtYmVmheH97tTlQ,crab-house-all-you-can-eat-seafood-new-york,Crab House All You Can Eat Seafood,https://s3-media2.fl.yelpcdn.com/bphoto/1J0qtJ...,False,https://www.yelp.com/biz/crab-house-all-you-ca...,863,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.5,"{'latitude': 40.759664, 'longitude': -73.969776}","[delivery, pickup]",$$$$,"{'address1': '135 E 55th St', 'address2': None...",+12129330264,(212) 933-0264,5918.197660
1,yrH_D2YMYIydwKk8MrVlqA,crab-house-brooklyn-all-you-can-eat-seafood-br...,Crab House Brooklyn All You Can Eat Seafood,https://s3-media4.fl.yelpcdn.com/bphoto/nLkqUV...,False,https://www.yelp.com/biz/crab-house-brooklyn-a...,85,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 40.57594204454698, 'longitude': -...",[],$$$$,"{'address1': '1223 Surf Ave', 'address2': None...",+13474621934,(347) 462-1934,14570.921975
2,kr_r3cmFlhgr_1jjLHMgfg,fulton-fish-new-york-3,Fulton Fish,https://s3-media1.fl.yelpcdn.com/bphoto/SlgSjd...,False,https://www.yelp.com/biz/fulton-fish-new-york-...,18,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.5,"{'latitude': 40.70623460823695, 'longitude': -...",[],$$$,"{'address1': '96 South St', 'address2': None, ...",+16468686000,(646) 868-6000,1950.561191
3,LpVbI-E9C-CSp7WhU5h2Iw,fishmarket-restaurant-new-york,Fishmarket Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/-rU1vu...,False,https://www.yelp.com/biz/fishmarket-restaurant...,632,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 40.70717, 'longitude': -74.00192}",[delivery],$$,"{'address1': '111 South St', 'address2': '', '...",+19173638101,(917) 363-8101,1914.204269
4,4H_wJKFIh7Xfu4OXQ-_6gg,the-monster-crab-bayside-new-york-2,The Monster Crab - Bayside,https://s3-media2.fl.yelpcdn.com/bphoto/3NonZD...,False,https://www.yelp.com/biz/the-monster-crab-bays...,62,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.5,"{'latitude': 40.77863165898115, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '211-35 26th Ave', 'address2': ''...",+17188198218,(718) 819-8218,18686.657387


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,C-k6jPOQEPJmhNUyCQDzlQ,fish-market-brooklyn-2,Fish Market,https://s3-media2.fl.yelpcdn.com/bphoto/h_vLK_...,False,https://www.yelp.com/biz/fish-market-brooklyn-...,6,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 40.686318, 'longitude': -73.915988}",[],$,"{'address1': '1551 Broadway', 'address2': '', ...",+17184533437,(718) 453-3437,5802.134543
996,7_b6dBRDIz9ovcWqq5bdkw,catchers-fish-house-freeport,Catcher's Fish House,https://s3-media1.fl.yelpcdn.com/bphoto/laaeYG...,False,https://www.yelp.com/biz/catchers-fish-house-f...,21,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"{'latitude': 40.631748, 'longitude': -73.581466}",[],NaN,"{'address1': '301 Woodlceft Ave', 'address2': ...",+15166009698,(516) 600-9698,34571.345824
997,HOA5esBzkJ_kPyjdxg_fFw,sayola-restaurant-tenafly,Sayola Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/pUeklg...,False,https://www.yelp.com/biz/sayola-restaurant-ten...,161,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,"{'latitude': 40.93116, 'longitude': -73.96111}","[pickup, delivery]",$$$,"{'address1': '50 Prospect Ter', 'address2': No...",+12018712182,(201) 871-2182,24973.251701
998,jjLcqGI11ThxAhcftEh1Aw,fraunces-tavern-new-york-6,Fraunces Tavern,https://s3-media3.fl.yelpcdn.com/bphoto/ayLfMY...,False,https://www.yelp.com/biz/fraunces-tavern-new-y...,1106,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 40.70344, 'longitude': -74.01135}","[pickup, delivery]",$$,"{'address1': '54 Pearl St', 'address2': '', 'a...",+12129681776,(212) 968-1776,2740.674551
999,F5YKnCOtmaDWLjL3Hu2tgQ,la-sala-by-tu-casa-kew-gardens-2,La Sala by Tu Casa,https://s3-media2.fl.yelpcdn.com/bphoto/z-Vwx5...,False,https://www.yelp.com/biz/la-sala-by-tu-casa-ke...,82,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 40.707369, 'longitude': -73.835057}",[],NaN,"{'address1': '11635 Metropolitan Ave', 'addres...",+17184873990,(718) 487-3990,12144.612829


In [13]:
# check for duplicate ID's 
seafood_df.duplicated(subset='id').sum()

198

In [14]:
## Drop duplicate ids and confirm there are no more duplicates
seafood_df = seafood_df.drop_duplicates(subset='id')
seafood_df.duplicated(subset='id').sum()

0